# 마인크래프트 분석 보고서
## 분석 주제: 오버월드에 있는 몹 중 가장 가성비 (잡는 난이도 대비 가치) 있는 몹은 무엇인가?

## 데이터 불러오기

In [1]:
# Initialise Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Minecraft').getOrCreate()
spark

In [2]:
# CSV 파일 로드

# 사냥 난이도 계산 파일
Mobs = spark.read.csv('../learning_spark_data/minecraft/Mobs.csv', header = True, inferSchema = True) # 몹 정보 불러오기

# 드롭 가치 계산 파일
## 음식
Food = spark.read.csv('../learning_spark_data/minecraft/Food.csv', header = True, inferSchema = True) # 음식 정보 불러오기
MobFood = spark.read.csv('../learning_spark_data/minecraft/MobFoodDrops.csv', header = True, inferSchema = True) # 몹이 드롭하는 음식 정보 불러오기

## Tool and Armour
TA = spark.read.csv('../learning_spark_data/minecraft/ToolsAndArmors.csv', header = True, inferSchema = True) # 도구 & 갑옷 정보 불러오기
MobTA = spark.read.csv('../learning_spark_data/minecraft/MobToolsAndArmorDrops.csv', header = True, inferSchema = True) # 몹이 드롭하는 도구 & 갑옷 정보 불러오기

In [3]:
# 데이터 확인
print(Mobs.show(10))
print(Food.show(10))
print(MobFood.show(10))
print(TA.show(10))
print(MobTA.show(10))

+---+--------------+-------------+---------------+------------+---------+-------------------+----------------+-----------------------+
| ID|          name|behaviorTypes|  spawnBehavior|healthPoints|maxDamage|          debutDate|minecraftVersion|reproductiveRequirement|
+---+--------------+-------------+---------------+------------+---------+-------------------+----------------+-----------------------+
|  1|   sniffer_egg|     Peaceful|Player Summoned|           0|        0|2023-06-07 00:00:00|          1.20.0|                   NULL|
|  2|    turtle_egg|     Peaceful|Player Summoned|           0|        0|2018-07-18 00:00:00|          1.13.0|                   NULL|
|  3|           bat|     Peaceful|         Random|           6|     NULL|2012-10-25 00:00:00|           1.4.2|                   NULL|
|  4|         blaze|      Hostile|        On Load|          20|        6|2011-11-18 00:00:00|           1.0.0|                   NULL|
|  5|       chicken|     Peaceful|         Random|     

In [4]:
# 필요없는 데이터 컬럼 drop
Mobs = Mobs.drop("behaviorTypes", "spawnBehavior", "debutDate", "minecraftVersion", "reproductiveRequirement")
Food = Food.drop("debutDate", "minecraftVersion")
TA = TA.drop("debutDate", "minecraftVersion")

불러온 데이터를 확인한 후 필요 없는 컬럼은 제거하였다.

Mobs에서는 "behaviorTypes", "spawnBehavior", "debutDate", "minecraftVersion", "reproductiveRequirement",\
Food에서는 "debutDate", "minecraftVersion",\
TA에서도 "debutDate", "minecraftVersion"를 드롭하였다.

In [5]:
# 드롭 후 데이터 확인
print(Mobs.show(10))
print(Food.show(10))
print(MobFood.show(10))
print(TA.show(10))
print(MobTA.show(10))

+---+--------------+------------+---------+
| ID|          name|healthPoints|maxDamage|
+---+--------------+------------+---------+
|  1|   sniffer_egg|           0|        0|
|  2|    turtle_egg|           0|        0|
|  3|           bat|           6|     NULL|
|  4|         blaze|          20|        6|
|  5|       chicken|           4|     NULL|
|  6|       dolphin|          10|        3|
|  7|     endermite|           8|        2|
|  8|      guardian|          30|        6|
|  9|guardian_elder|          80|        8|
| 10|       phantom|          20|        2|
+---+--------------+------------+---------+
only showing top 10 rows

None
+---+---------------+------+------+
| ID|           name|  type|hunger|
+---+---------------+------+------+
| 13| cooked_chicken|Animal|     6|
| 19|         cookie| Plant|     2|
|  1|          apple| Plant|     4|
| 25|    melon_slice| Plant|     2|
| 26|    milk_bucket|Animal|     0|
| 27|  mushroom_stew| Plant|     6|
| 16|cooked_porkchop|Animal| 

In [6]:
# 필요없는 행 삭제
from pyspark.sql.functions import col

Mobs = Mobs.filter(col("healthPoints") != 0)

Mobs 데이터에서 healthPoint가 0인 몹들은 식량이 될 수 없기 때문에 생존에 도움이 되지 않는다. 따라서 제거했다.

In [7]:
spark.stop()